# **Building_Dating_App_Recommendation_System**

These recommendations are based on various factors and preferences, aiming to increase the likelihood of successful and compatible matches. Accurate dating recommendations enhance the user experience by providing personalized dating recommendations to facilitate meaningful connections. We have access to a dataset containing user profiles with various attributes such as age, gender, height, interests, education level, occupation, and more. These profiles are associated with user behaviours, including swiping history and frequency of app usage.

The dataset consists of the following columns:

- User ID: A unique identifier for each user.
- Age: The age of the user.
- Gender: The gender of the user (e.g., Male, Female).
- Height: The height of the user.
- Interests: A list of interests or hobbies expressed by the user.
- Looking For: The user’s dating preferences, such as “Long-term Relationship,” “Marriage,” etc.
- Children: Indicates whether the user has children (“Yes,” “No,” “Maybe”).
- Education Level: The user’s highest education level.
- Occupation: The user’s current occupation.
- Swiping History: A numerical score representing the user’s interactions (e.g., likes/dislikes) with other profiles.
- Frequency of Usage: How often the user engages with the dating app (e.g., “Daily,” “Monthly”).

Dating recommendations involve a combination of data collection, preprocessing, and user profiling techniques to provide tailored matchmaking suggestions to users of dating platforms. The goal is to enhance user experiences by increasing the chances of meaningful and compatible connections.

Here are the steps we can follow for the task of dating recommendations:

Gather data from the dating platform, including user profiles, preferences, past interactions, and feedback.
Clean and preprocess the collected data.
Create meaningful features from the raw data. For example, you might extract features like age, location, interests, and compatibility scores from user profiles.
Develop and recommend user profiles by analyzing user behaviour, such as swipes, likes, and messaging patterns.

The goal of this task is to design and implement a data-driven matchmaking and recommendation system for the dating app. The objective is to generate meaningful dating recommendations by considering user-profiles and behaviours while respecting their preferences.



### **Setup**

In [1]:
# Load necessary libraries
import pandas as pd
import plotly.express as px

%matplotlib notebook

In [2]:
# Load data
data = pd.read_csv('dating_app_dataset.csv')

In [3]:
data.head(10)

,User ID,Age,Gender,Height,Interests,Looking For,Children,Education Level,Occupation,Swiping History,Frequency of Usage
0,1,30,Male,5.240385,"['Sports', 'Cooking', 'Hiking', 'Music', 'Movi...",Casual Dating,No,High School,Student,96,Weekly
1,2,27,Female,4.937625,"['Sports', 'Reading']",Friendship,Yes,Master's Degree,Artist,96,Monthly
2,3,29,Female,5.806296,['Sports'],Casual Dating,No,Bachelor's Degree,Social Media Influencer,64,Daily
3,4,29,Female,5.101402,['Reading'],Marriage,No,Ph.D.,Doctor,67,Daily
4,5,32,Male,5.986670,"['Sports', 'Hiking', 'Music', 'Movies', 'Readi...",Long-term Relationship,Yes,Ph.D.,Engineer,93,Monthly
5,6,25,Female,5.192045,"['Sports', 'Cooking', 'Movies', 'Music', 'Trav...",Marriage,Yes,Master's Degree,Engineer,38,Monthly
6,7,27,Female,5.725126,"['Travel', 'Cooking']",Marriage,No,Master's Degree,Student,77,Monthly
7,8,21,Male,5.108276,"['Cooking', 'Hiking', 'Movies', 'Reading', 'Mu...",Marriage,Yes,Bachelor's Degree,Teacher,87,Daily
8,9,33,Female,5.364951,"['Music', 'Reading', 'Sports', 'Cooking', 'Mov...",Friendship,Maybe,High School,Social Media Influencer,6,Monthly
9,10,26,Female,6.020815,"['Sports', 'Reading', 'Music', 'Cooking', 'Mov...",Long-term Relationship,No,Ph.D.,Teacher,8,Weekly


In [4]:
print(data.isnull().sum())

User ID               0
Age                   0
Gender                0
Height                0
Interests             0
Looking For           0
Children              0
Education Level       0
Occupation            0
Swiping History       0
Frequency of Usage    0
dtype: int64


In [5]:
# Get to know data type, information and statistic descriptive
print(data.dtypes)
print(data.info())
print(data.describe())

User ID                 int64
Age                     int64
Gender                 object
Height                float64
Interests              object
Looking For            object
Children               object
Education Level        object
Occupation             object
Swiping History         int64
Frequency of Usage     object
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   User ID             500 non-null    int64  
 1   Age                 500 non-null    int64  
 2   Gender              500 non-null    object 
 3   Height              500 non-null    float64
 4   Interests           500 non-null    object 
 5   Looking For         500 non-null    object 
 6   Children            500 non-null    object 
 7   Education Level     500 non-null    object 
 8   Occupation          500 non-null    object 
 9   Swiping

### Data Exploration

In this step we are going to do some exploration for better understanding about our data before creating recommendation system. Here's to explore the age distribution by gender, education level by gender, and frequency app usage by gender.

In [6]:
# Age distribution by gender
fig1 = px.histogram(data, x='Age', color='Gender',
                   nbins=20, title='Age Distribution by Gender')
fig1.update_layout(xaxis_title='Age', yaxis_title='Count')
fig1.show()

In [7]:
# Education level by gender
education_order = ['High School', "Bachelor's Degree", "Master's Degree", 'Ph.D.']
fig2 = px.bar(data, x='Education Level', color='Gender',
             category_orders={'Education Level': education_order},
             title='Education Level Distribution by Gender')

fig2.update_layout(xaxis_title='Education Level', yaxis_title='Count')
fig2.show()

In [8]:
# distribution of frequency using app by gender
fig3 = px.bar(data, x='Frequency of Usage', color='Gender',
             title='Frequency of App Usage Distribution by Gender')

fig3.update_layout(xaxis_title='Frequency of Usage', yaxis_title='Count')
fig3.show()

After gained informations about distribution of each features, we want to separate data into respective gender that is **Male** and **Female**.

In [9]:
# Separate data into male and female
male_user = data[data['Gender'] == 'Male']
female_user = data[data['Gender'] == 'Female']

In [10]:
# Here's to calculate the match score for dating recommendation
def calculate_match_score(profile1, profile2):
    # Shared interest score (1 point per share)
    interests1 = set(eval(profile1['Interests']))
    interests2 = set(eval(profile2['Interests']))
    shared_interests_score = len(interests1.intersection(interests2))

    # Age difference score (higher age difference, lower score)
    age_difference_score = max(0, 10 - abs(profile1['Age'] - profile2['Age']))

    # Swiping history score (higher swipping history, higher score)
    swiping_history_score = min(profile1['Swiping History'], profile2['Swiping History']) / 100

    # Relationship type score (1 point for matching type)
    relationship_type_score = 0
    if profile1['Looking For'] == profile2['Looking For']:
        relationship_type_score = 1

    # Total match score
    total_score = (
        shared_interests_score + age_difference_score + swiping_history_score + relationship_type_score
    )

    return total_score

# Example: Calculate match score between two profiles
profile1 = male_user.iloc[0]
profile2 = female_user.iloc[0]
match_score = calculate_match_score(profile1, profile2)

# Show result
print(f"Match score between User {profile1['User ID']} and User {profile2['User ID']} : {match_score}")


Match score between User 1 and User 2 : 9.96


The goal of the Python script is to calculate the compatibility score between two user profiles on a dating app. This compatibility score can be used to recommend other users who may be compatible with the current user.

This script works by separating user data into two groups based on gender, namely men and women. Then, define a function called calculate_match_score(). This function calculates a compatibility score between two user profiles based on the following factors:

Shared interest score (1 point per share): This score is calculated based on the number of interests shared between two users.
1. Age difference score (higher age difference, lower score): This score is calculated based on the age difference between two users. The greater the age difference, the lower the compatibility score.
2. Swiping history score (higher swiping history, higher score): This score is calculated based on the swiping history of two users. The higher the swiping history, the higher the compatibility score.
3. Relationship type score (1 point for matching type): This score is calculated based on the suitability of the type of relationship desired by two users. If two users want the same type of relationship, then their compatibility score will be increased by 1.

Finally, this script shows an example of using the calculate_match_score() function to calculate the compatibility score between the first two users in each group.

Here’s how to generate dating recommendations by utilizing the match score we calculated above.

In [11]:
def recommend_profiles(male_user, female_user):
    recommendations = []

    for _, male_profile in male_user.iterrows():
        best_match = None
        best_score = -1

        for _, female_profile in female_user.iterrows():
            score = calculate_match_score(male_profile, female_profile)

            if score > best_score:
                best_match = female_profile
                best_score = score

            recommendations.append((male_profile, best_match, best_score))

    return recommendations

# Generate recommendations
recommendations = recommend_profiles(male_user, female_user)

# Sort recommendations by match score in descending order
recommendations.sort(key=lambda x: x[2], reverse=True)

# Display the top recommendations
for idx, (male_profile, female_profile, match_score) in enumerate(recommendations[:10]):
    print(f"Recommendation {idx + 1}:")
    print(f"Male Profile (User {male_profile['User ID']}): Age {male_profile['Age']}, Interests {male_profile['Interests']}")
    print(f"Female Profile (User {female_profile['User ID']}): Age {female_profile['Age']}, Interests {female_profile['Interests']}")
    print(f"Match Score: {match_score}")
    print()

Recommendation 1:
Male Profile (User 36): Age 19, Interests ['Movies', 'Cooking', 'Hiking', 'Reading', 'Sports', 'Travel', 'Music']
Female Profile (User 451): Age 19, Interests ['Reading', 'Music', 'Cooking', 'Hiking', 'Travel', 'Sports', 'Movies']
Match Score: 18.79

Recommendation 2:
Male Profile (User 36): Age 19, Interests ['Movies', 'Cooking', 'Hiking', 'Reading', 'Sports', 'Travel', 'Music']
Female Profile (User 451): Age 19, Interests ['Reading', 'Music', 'Cooking', 'Hiking', 'Travel', 'Sports', 'Movies']
Match Score: 18.79

Recommendation 3:
Male Profile (User 36): Age 19, Interests ['Movies', 'Cooking', 'Hiking', 'Reading', 'Sports', 'Travel', 'Music']
Female Profile (User 451): Age 19, Interests ['Reading', 'Music', 'Cooking', 'Hiking', 'Travel', 'Sports', 'Movies']
Match Score: 18.79

Recommendation 4:
Male Profile (User 36): Age 19, Interests ['Movies', 'Cooking', 'Hiking', 'Reading', 'Sports', 'Travel', 'Music']
Female Profile (User 451): Age 19, Interests ['Reading', 'Mus

In the above code, we are creating a recommendation system that suggests potential matches between male and female user profiles. The recommendation system evaluates the compatibility of these profiles based on various factors, including shared interests, age, and other attributes we used to calculate the match score.

The first script, recommend_profiles(), takes two arguments: male_user and female_user. These arguments are DataFrames that contain the user profiles for male and female users, respectively.

The function works by iterating through the male users and calculating their match score with each female user. The match score is calculated using the calculate_match_score() function from the previous script.

Once the match score for all female users has been calculated, the function selects the female user with the highest match score and adds the pair to a list of recommendations.

The second script, generate_recommendations(), simply calls the recommend_profiles() function and stores the results in a variable called recommendations.

The third script, sort_recommendations(), sorts the recommendations list in descending order by match score.

The fourth script, display_recommendations(), displays the top 10 recommendations to the user.
